In [2]:
# السكربت الثاني: face_recognition من أجل التعرف على وجه الطفل
# هذا الملف يجب تشغيله في بيئة بايثون 3.10 فقط (virtualenv)

import face_recognition
import cv2
import os
import time
import winsound
import json

# مسار صورة وجه الطفل
known_image_path = "child_photos/photo_3.jpg"  # عدلي مسار صورة ابن المستخدم لاحقًا

print("🧒 تحميل صورة الطفل")
known_image = face_recognition.load_image_file(known_image_path)
known_encodings = face_recognition.face_encodings(known_image)
if not known_encodings:
    print("❌ لم يتم اكتشاف وجه في صورة الطفل!")
    exit()

known_encoding = known_encodings[0]

# قراءة فريمات محفوظة (yolo يضعها في input_frames)
frames_dir = "input_frames"
frame_files = sorted(os.listdir(frames_dir))

print("🎥 تشغيل المراقبة على الفريمات")

for frame_file in frame_files:
    frame_path = os.path.join(frames_dir, frame_file)
    frame = cv2.imread(frame_path)
    if frame is None:
        print(f"❌ فشل في قراءة: {frame_path}")
        continue

    rgb_frame = frame[:, :, ::-1]

    # كشف الوجوه وتمييزها
    locations = face_recognition.face_locations(rgb_frame)
    encodings = face_recognition.face_encodings(rgb_frame, locations)

    child_detected = False
    for encoding in encodings:
        match = face_recognition.compare_faces([known_encoding], encoding, tolerance=0.45)[0]
        if match:
            child_detected = True
            break

    if child_detected:
        # قراءة الحالة من JSON لرؤية ماذا يكشف YOLO
        try:
            with open("detection_state.json", "r") as f:
                state = json.load(f)
                if state.get("sharp_object_detected"):
                    print("⚠️ تنبيه: الطفل قريب من أداة حادة!")
                    winsound.Beep(1000, 500)
        except Exception as e:
            print("❌ خطأ في قراءة detection_state.json", e)

    time.sleep(1)


🧒 تحميل صورة الطفل
🎥 تشغيل المراقبة على الفريمات


KeyboardInterrupt: 